# 🥈 Camada Silver

Nesta camada Silver do pipeline de dados, realizamos a transformação, padronização e enriquecimento dos dados brutos provenientes da camada Bronze, preparando-os para análises mais avançadas e para o consumo por outras áreas do negócio. Ela será fundamental para garantir a qualidade e confiabilidade dos dados, promovendo uma cultura data-driven e apoiando a tomada de decisão baseada em informações sólidas.

## Principais Atividades Realizadas

- **Padronização de Identificadores:** Todos os identificadores de pedidos, clientes, produtos e vendedores foram normalizados (removendo espaços, convertendo para minúsculas ou maiúsculas conforme necessário) para garantir consistência e facilitar integrações entre tabelas.

- **Tratamento de Datas e Horários:** As datas e horários relevantes (compra, aprovação, entrega, etc.) foram convertidos para o fuso horário de 'America/Sao_Paulo' e formatados de maneira padronizada, permitindo análises temporais precisas.

- **Normalização de Status e Tipos:** Os status dos pedidos e tipos de pagamento passaram por tradução automática para o português, remoção de acentuação e padronização em letras maiúsculas, garantindo uniformidade e eliminando ambiguidades.

- **Conversão e Tipagem de Dados:** Valores numéricos, como parcelas, preços e valores de frete, foram convertidos para tipos apropriados (INT, FLOAT, DECIMAL), assegurando precisão e facilitando cálculos futuros.

- **Criação de Tabelas Temáticas:** Foram criadas tabelas intermediárias e finais para pedidos (`100cep_pedidos`), pagamentos (`100cep_pagamentos`), itens dos pedidos (`100cep_itens_pedidos`) e clientes (`100cep_clientes`), cada uma com os campos já tratados e prontos para análise.

- **Enriquecimento dos Dados:** Além da limpeza, foram realizados enriquecimentos como a extração de prefixos de CEP dos clientes, o que pode ser útil para análises geográficas e segmentações.

## Benefícios da Camada Silver

- **Dados Prontos para Análise:** Os dados estão limpos, padronizados e organizados, prontos para serem consumidos por dashboards, relatórios e modelos analíticos.
- **Facilidade de Integração:** A padronização dos campos facilita o cruzamento de informações entre diferentes tabelas e fontes.
- **Redução de Erros:** O tratamento prévio dos dados minimiza inconsistências e potenciais erros em análises futuras.
- **Base para a Camada Gold:** Esta camada serve como base confiável para a construção de métricas, indicadores e visões de negócio na camada Gold.

In [0]:
%sql
USE CATALOG `100cep_gateway`;

In [0]:
%sql
USE SCHEMA silver; 

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_pedidos AS
WITH bronze_order_normalized AS (
  SELECT 
    TRIM(LOWER(order_id)) AS pedido_id,
    TRIM(LOWER(customer_id)) AS cliente_id,
    TRIM(
      UPPER(
        TRANSLATE(
          AI_TRANSLATE(order_status, 
            'portuguese'),
            'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ', 
            'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
        )
      )
    ) AS status_pedido,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_purchase_timestamp :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS data_pedido,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_approved_at :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS aprovacao_pedido,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_delivered_carrier_date :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS data_entrega_transportadora,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_delivered_customer_date :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS data_entrega_cliente,
    order_estimated_delivery_date :: DATE AS data_entrega_estimada
  FROM 
    100cep_gateway.bronze.100cep_orders_raw
)
SELECT
  *
FROM 
  bronze_order_normalized;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
  *
FROM
  100cep_pedidos
LIMIT 10;

pedido_id,cliente_id,status_pedido,data_pedido,aprovacao_pedido,data_entrega_transportadora,data_entrega_cliente,data_entrega_estimada
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,ENTREGUE,2017-10-02 07:56:33,2017-10-02 08:07:15,2017-10-04 16:55:00,2017-10-10 18:25:13,2017-10-18
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,ENTREGUE,2018-07-24 17:41:37,2018-07-26 00:24:27,2018-07-26 11:31:00,2018-08-07 12:27:45,2018-08-13
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,ENTREGUE,2018-08-08 05:38:49,2018-08-08 05:55:23,2018-08-08 10:50:00,2018-08-17 15:06:29,2018-09-04
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,ENTREGUE,2017-11-18 17:28:06,2017-11-18 17:45:59,2017-11-22 11:39:59,2017-12-01 22:28:42,2017-12-15
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,ENTREGUE,2018-02-13 19:18:39,2018-02-13 20:20:29,2018-02-14 17:46:34,2018-02-16 16:17:02,2018-02-26
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,ENTREGUE,2017-07-09 18:57:05,2017-07-09 19:10:13,2017-07-11 11:58:04,2017-07-26 07:57:55,2017-08-01
136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,FATURADO,2017-04-11 09:22:08,2017-04-13 10:25:17,null,null,2017-05-09
6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,ENTREGUE,2017-05-16 10:10:30,2017-05-16 10:22:11,2017-05-22 07:07:46,2017-05-26 09:55:51,2017-06-07
76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,ENTREGUE,2017-01-23 16:29:09,2017-01-25 00:50:47,2017-01-26 12:16:31,2017-02-02 12:08:10,2017-03-06
e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,ENTREGUE,2017-07-29 08:55:02,2017-07-29 09:05:32,2017-08-10 16:45:24,2017-08-16 14:14:30,2017-08-23


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_pagamentos AS
WITH bronze_order_payments_normalized AS (
  SELECT 
    order_id AS pedido_id,
    CAST(payment_sequential AS INT) AS pagamento_sequencial,
    TRIM(
      UPPER(
        TRANSLATE(
          AI_TRANSLATE(
            REPLACE(payment_type, '_', ' '), 
            'portuguese'),
          'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
          'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
        )
      )
    ) AS tipo_pagamento,
    CAST(payment_installments AS INT) AS pagamento_parcelas,
    CAST(ROUND(payment_value, 2) AS FLOAT) AS valor_transacao
  FROM 
    100cep_gateway.bronze.100cep_order_payments_raw
)
SELECT 
  *
FROM 
  bronze_order_payments_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
  *
FROM
  100cep_pagamentos
LIMIT 10;

pedido_id,pagamento_sequencial,tipo_pagamento,pagamento_parcelas,valor_transacao
b81ef226f3fe1789b1e8b2acac839d17,1,CARTAO DE CREDITO,8,99.33
a9810da82917af2d9aefd1278f1dcfa0,1,CARTAO DE CREDITO,1,24.39
25e8ea4e93396b6fa0d3dd708e76c1bd,1,CARTAO DE CREDITO,1,65.71
ba78997921bbcdc1373bb41e913ab953,1,CARTAO DE CREDITO,8,107.78
42fdf880ba16b47b59251dd489d4441a,1,CARTAO DE CREDITO,2,128.45
298fcdf1f73eb413e4d26d01b25bc1cd,1,CARTAO DE CREDITO,2,96.12
771ee386b001f06208a7419e4fc1bbd7,1,CARTAO DE CREDITO,1,81.16
3d7239c394a212faae122962df514ac7,1,CARTAO DE CREDITO,3,51.84
1f78449c87a54faf9e96e88ba1491fa9,1,CARTAO DE CREDITO,6,341.09
0573b5e23cbd798006520e1d5b4c6714,1,BOLETO,1,51.95


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_itens_pedidos AS
WITH bronze_order_items_normalized AS (
SELECT 
  order_id AS pedido_id,
  order_item_id AS item_pedido_id,
  product_id AS produto_id,
  seller_id AS vendedor_id,
  CAST(shipping_limit_date AS DATE) AS data_entrega_limite,
  price :: DECIMAL (6, 2) AS preco_produto,
  freight_value :: DECIMAL(5, 2) AS valor_frete
FROM 
  100cep_gateway.bronze.`100cep_order_items_raw`
) 
SELECT 
  * 
FROM 
  bronze_order_items_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
  *
FROM
  100cep_itens_pedidos
LIMIT 10;

pedido_id,item_pedido_id,produto_id,vendedor_id,data_entrega_limite,preco_produto,valor_frete
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19,58.90,13.29
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03,239.90,19.93
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18,199.00,17.87
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13,199.90,18.14
00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,2017-05-23,21.90,12.69
00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,7040e82f899a04d1b434b795a43b4617,2017-12-14,19.90,11.85
000576fe39319847cbb9d288c5617fa6,1,557d850972a7d6f792fd18ae1400d9b6,5996cddab893a4652a15592fb58ab8db,2018-07-10,810.00,70.75
0005a1a1728c9d785b8e2b08b904576c,1,310ae3c140ff94b03219ad0adc3c778f,a416b6a846a11724393025641d4edd5e,2018-03-26,145.95,11.65
0005f50442cb953dcd1d21e1fb923495,1,4535b0e1091c278dfd193e5a1d63b39f,ba143b05f0110f0dc71ad71b4466ce92,2018-07-06,53.99,11.40


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_clientes AS
WITH bronze_customers_normalized AS (
  SELECT 
    customer_id AS cliente_id,
    customer_unique_id AS cliente_id_unico,
    customer_zip_code_prefix AS cep_prefixo
  FROM 
    100cep_gateway.bronze.100cep_customers_raw
)
SELECT 
  *
FROM 
  bronze_customers_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT 
  *
FROM 
  100cep_clientes
LIMIT 10;

cliente_id,cliente_id_unico,cep_prefixo
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056
879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254
fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,04534
5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182
5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560
4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_vendedores AS
WITH bronze_sellers_normalized AS (
SELECT 
  seller_id AS vendedor_id,
  seller_zip_code_prefix AS cep_prefixo
FROM 
  `100cep_gateway`.bronze.`100cep_sellers_raw`
)
SELECT 
  *
FROM
  bronze_sellers_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT 
  *
FROM 
  100cep_vendedores
LIMIT 10;

vendedor_id,cep_prefixo
3442f8959a84dea7ee197c632cb2df15,13023
d1b65fc7debc3361ea86b5f14c68d2e2,13844
ce3ad9de960102d0677a81f5d0bb7b2d,20031
c0f3eea2e14555b6faeea3dd58c1b1c3,04195
51a04a8a6bdcb23deccc82b0b80742cf,12914
c240c4061717ac1806ae6ee72be3533b,20920
e49c26c3edfa46d227d5121a6b6e4d37,55325
1b938a7ec6ac5061a66a3766e0e75f90,16304
768a86e36ad6aae3d03ee3c6433d61df,01529
ccc4bbb5f32a6ab2b7066a4130f114e3,80310


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_avaliacoes AS
WITH bronze_order_reviews_normalized AS (
SELECT
   TRIM(LOWER(
       CASE WHEN LENGTH(review_id) = 32 AND review_id NOT LIKE '% %' 
            THEN review_id END
   )) AS avaliacao_id,

   TRIM(LOWER(
       CASE WHEN LENGTH(order_id) = 32 AND order_id NOT LIKE '% %' 
            THEN order_id END
   )) AS pedido_id,

   review_score AS avaliacao_nota,

   TRIM(UPPER(
        CASE 
            WHEN AI_ANALYZE_SENTIMENT(review_comment_message) = 'positive' THEN 'positivo'
            WHEN AI_ANALYZE_SENTIMENT(review_comment_message) = 'negative' THEN 'negativo'
            WHEN AI_ANALYZE_SENTIMENT(review_comment_message) = 'neutral' THEN 'neutro'
            ELSE 'desconhecido'
        END
   )) AS `avaliacao_sentimento`,

   TRY_CAST(
       CASE WHEN LENGTH(review_creation_date) = 19 
            THEN review_creation_date END
       AS DATE
   ) AS avaliacao_data,

   review_answer_timestamp AS avaliacao_hora_resposta
FROM
  `100cep_gateway`.bronze.`100cep_order_reviews_raw`
WHERE 
    (LENGTH(review_id) = 32 AND review_id NOT LIKE '% %')
 OR (LENGTH(order_id) = 32 AND order_id NOT LIKE '% %')
)
SELECT 
  *
FROM 
  bronze_order_reviews_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT 
  *
FROM 
  100cep_avaliacoes
LIMIT 10;

avaliacao_id,pedido_id,avaliacao_nota,avaliacao_sentimento,avaliacao_data,avaliacao_hora_resposta
7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,DESCONHECIDO,2018-01-18,2018-01-18 21:46:59
80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,DESCONHECIDO,2018-03-10,2018-03-11 03:05:13
228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,DESCONHECIDO,2018-02-17,2018-02-18 14:36:24
e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,POSITIVO,2017-04-21,2017-04-21 22:02:06
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,POSITIVO,2018-03-01,2018-03-02 10:26:53
15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,DESCONHECIDO,2018-04-13,2018-04-16 00:39:37
07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,DESCONHECIDO,2017-07-16,2017-07-18 19:30:34
7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,DESCONHECIDO,2018-08-14,2018-08-14 21:36:06
a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,DESCONHECIDO,2017-05-17,2017-05-18 12:05:37
8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,DESCONHECIDO,2018-05-22,2018-05-23 16:45:47


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_categoria_produtos AS
WITH bronze_product_category_normalized AS (
SELECT 
    TRIM(UPPER(TRANSLATE(
        product_category_name,
        'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
        'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
      ))) AS categoria_produto,
    TRIM(UPPER(TRANSLATE(
        product_category_name_english,
        'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
        'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
      ))) AS categoria_produto_ingles
FROM `100cep_gateway`.bronze.`100cep_product_category_name_translation_raw`
)
SELECT 
    *
FROM 
    bronze_product_category_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT 
  *
FROM 
  100cep_categoria_produtos
LIMIT 10;

categoria_produto,categoria_produto_ingles
BELEZA_SAUDE,HEALTH_BEAUTY
INFORMATICA_ACESSORIOS,COMPUTERS_ACCESSORIES
AUTOMOTIVO,AUTO
CAMA_MESA_BANHO,BED_BATH_TABLE
MOVEIS_DECORACAO,FURNITURE_DECOR
ESPORTE_LAZER,SPORTS_LEISURE
PERFUMARIA,PERFUMERY
UTILIDADES_DOMESTICAS,HOUSEWARES
TELEFONIA,TELEPHONY
RELOGIOS_PRESENTES,WATCHES_GIFTS


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_produtos AS 
WITH bronze_products_normalized AS (
SELECT 
  product_id AS produto_id,
  TRIM(UPPER(TRANSLATE(
        product_category_name,
        'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
        'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
      ))) AS categoria_produto,
  product_name_lenght AS nome_produto_tamanho,
  product_description_lenght AS descricao_produto_tamanho,
  product_photos_qty AS fotos_produto_quantidade,
  product_weight_g AS peso_produto,
  product_length_cm AS comprimento_produto,
  product_height_cm AS altura_produto,
  product_width_cm AS largura_produto
FROM 
  `100cep_gateway`.bronze.`100cep_products_raw`
)
SELECT 
  *
FROM 
  bronze_products_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT 
  *
FROM 
  100cep_produtos
LIMIT 10;

produto_id,categoria_produto,nome_produto_tamanho,descricao_produto_tamanho,fotos_produto_quantidade,peso_produto,comprimento_produto,altura_produto,largura_produto
1e9e8ef04dbcff4541ed26657ea517e5,PERFUMARIA,40,287,1,225,16,10,14
3aa071139cb16b67ca9e5dea641aaa2f,ARTES,44,276,1,1000,30,18,20
96bd76ec8810374ed1b65e291975717f,ESPORTE_LAZER,46,250,1,154,18,9,15
cef67bcfe19066a932b7673e239eb23d,BEBES,27,261,1,371,26,4,26
9dc1a7de274444849c219cff195d0b71,UTILIDADES_DOMESTICAS,37,402,4,625,20,17,13
41d3672d4792049fa1779bb35283ed13,INSTRUMENTOS_MUSICAIS,60,745,1,200,38,5,11
732bd381ad09e530fe0a5f457d81becb,COOL_STUFF,56,1272,4,18350,70,24,44
2548af3e6e77a690cf3eb6368e9ab61e,MOVEIS_DECORACAO,56,184,2,900,40,8,40
37cc742be07708b53a98702e77a21a02,ELETRODOMESTICOS,57,163,1,400,27,13,17
8c92109888e8cdf9d66dc7e463025574,BRINQUEDOS,36,1156,1,600,17,10,12


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_geolocalizacao AS
WITH geolocation_normalized AS (
SELECT
geolocation_zip_code_prefix AS cep_prefixo,
  geolocation_lat AS latitude,
  geolocation_lng AS longitude,
  TRIM(
    UPPER(
      REGEXP_REPLACE(
        TRANSLATE(
          geolocation_city,
          'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
          'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
        ),
        '[^A-Za-z0-9 ]',
        ''
      )
    )
  ) AS cidade,
  TRIM(UPPER(geolocation_state)) AS estado
FROM 
  `100cep_gateway`.bronze.`100cep_geolocation_raw`
ORDER BY
  cidade
)
SELECT 
  *
FROM 
  geolocation_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT 
  *
FROM 
  100cep_geolocalizacao
LIMIT 10;

cep_prefixo,latitude,longitude,cidade,estado
87365,-24.28190538509627,-53.07451607086753,4 CENTENARIO,PR
87365,-24.27769659953568,-53.07493038222118,4 CENTENARIO,PR
87365,-24.2718596772122,-53.0694333134515,4O CENTENARIO,PR
75345,-16.76645713323428,-49.43420978223543,ABADIA DE GOIAS,GO
75345,-16.765307561956746,-49.429008193096806,ABADIA DE GOIAS,GO
75345,-16.766377164880446,-49.43746689694972,ABADIA DE GOIAS,GO
75345,-16.766377164880446,-49.43746689694972,ABADIA DE GOIAS,GO
75345,-16.756821000000002,-49.43502000000001,ABADIA DE GOIAS,GO
75345,-16.789538999999944,-49.46943049999996,ABADIA DE GOIAS,GO
75345,-16.767622962030664,-49.42667354436809,ABADIA DE GOIAS,GO


In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_chargebacks
WITH bronze_chargebacks_normalized AS (
SELECT 
  chargeback_id AS pedido_id,
  TRIM(UPPER(chargeback_reason)) AS motivo_chargeback,
  TRIM(UPPER(chargeback_status)) AS status_chargeback,
  chargeback_amount :: DECIMAL(6,2) AS valor_chargeback,
  TRIM(UPPER(issuer_response)) AS resposta_emitente,
  TRIM(UPPER(REPLACE(acquirer_response, "disptutando", "disputando"))) AS resposta_adquirente
FROM `100cep_gateway`.bronze.`100cep_chargebacks_raw`
WHERE chargeback_id IS NOT NULL
)
SELECT 
  *
FROM 
  bronze_chargebacks_normalized;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT 
  * 
FROM 
  100cep_chargebacks
LIMIT 10

chargeback_id,motivo_chargeback,status_chargeback,valor_chargeback,resposta_emitente,resposta_adquirente
c8c528189310eaa44a745b8d9d26908b,FRAUDE,GANHO,0.00,APROVADO,ACEITO
1dacb75a3f524523a6521835a6787057,VOUCHER_INDISPONIVEL,GANHO,100.00,APROVADO,ACEITO
f33aa647f08ec9623c76b6212be92333,VOUCHER_INDISPONIVEL,GANHO,100.34,APROVADO,ACEITO
ba3fbd2edbc0524c198765b79c67875d,MERCADORIA_NAO_RECEBIDA,GANHO,1008.95,APROVADO,ACEITO
cc0a0d27ed6897ad08f28a1f2472a49d,MERCADORIA_NAO_RECEBIDA,GANHO,101.14,APROVADO,ACEITO
7701ecd93e987006334fba168c936f20,FRAUDE,GANHO,101.56,APROVADO,ACEITO
53a0b865b63b4a2b64f8f46b31cdf3ed,ERRO_DE_PROCESSAMENTO,GANHO,102.01,APROVADO,ACEITO
d01a489bb718b638df1e789951d37fbb,MERCADORIA_NAO_RECEBIDA,GANHO,102.26,APROVADO,ACEITO
546c8958ff67b28473cf140ec32a5a91,ERRO_DE_PROCESSAMENTO,GANHO,102.56,APROVADO,ACEITO
cf8ba1509b00eca37b08289371dbbf0d,MERCADORIA_NAO_RECEBIDA,GANHO,102.64,APROVADO,ACEITO


In [0]:
%sql
SHOW TABLES

database,tableName,isTemporary
silver,100cep_avaliacoes,false
silver,100cep_categoria_produtos,false
silver,100cep_chargebacks,false
silver,100cep_clientes,false
silver,100cep_geolocalizacao,false
silver,100cep_itens_pedidos,false
silver,100cep_pagamentos,false
silver,100cep_pedidos,false
silver,100cep_produtos,false
silver,100cep_vendedores,false


In [0]:
%sql
SELECT 'avaliacoes' AS table_name, COUNT(*) AS n_linhas FROM 100cep_avaliacoes
UNION ALL
SELECT 'categoria_produtos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_categoria_produtos
UNION ALL
SELECT 'clientes' AS table_name, COUNT(*) AS n_linhas FROM 100cep_clientes
UNION ALL
SELECT 'geolocalizacao' AS table_name, COUNT(*) AS n_linhas FROM 100cep_geolocalizacao
UNION ALL
SELECT 'itens_pedidos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_itens_pedidos
UNION ALL
SELECT 'pagamentos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_pagamentos
UNION ALL
SELECT 'pedidos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_pedidos
UNION ALL
SELECT 'produtos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_produtos
UNION ALL
SELECT 'vendedores' AS table_name, COUNT(*) AS n_linhas FROM 100cep_vendedores
UNION ALL
SELECT 'chargebacks' AS table_name, COUNT(*) AS n_linhas FROM 100cep_chargebacks;

table_name,n_linhas
avaliacoes,99224
categoria_produtos,71
clientes,99441
geolocalizacao,1000163
itens_pedidos,112650
pagamentos,103886
pedidos,99441
produtos,32951
vendedores,3095
chargebacks,3238
